In [5]:
import statsmodels.api as sm
import numpy as np
import pandas as pd

from fredapi import Fred
from dotenv import load_dotenv
import os
load_dotenv()  # take environment variables
FREDAPIKEY = os.getenv('FREDAPIKEY')
fred = Fred(api_key=FREDAPIKEY)

In [15]:
gdp = fred.get_series_latest_release('GDPC1').rename("GDPC1")
pce = fred.get_series_latest_release('PCECC96').rename("PCECC96")

In [16]:
# Prepare your data
# X should be your independent variables (add constant if needed)
# y should be your dependent variable
X = sm.add_constant(pce)  # Add constant term
y = gdp.copy()


In [17]:
# Fit OLS regression
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  GDPC1   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 2.640e+05
Date:                Thu, 19 Dec 2024   Prob (F-statistic):               0.00
Time:                        19:39:24   Log-Likelihood:                -2105.4
No. Observations:                 311   AIC:                             4215.
Df Residuals:                     309   BIC:                             4222.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        602.3464     22.041     27.329      0.0

In [21]:
# Now with HAC standard errors
model = sm.OLS(y, X)

# Calculate optimal lag length
T = len(y)  # Sample size
optimal_lag = int(np.floor(4 * (T/100)**(2/9))) # Newey and West, 1994, "Automatic Lag Selection in Covariance Matrix Estimation," RES

results = model.fit(cov_type='HAC', cov_kwds={'maxlags': optimal_lag, 'use_correction': True})

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  GDPC1   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 3.626e+04
Date:                Thu, 19 Dec 2024   Prob (F-statistic):          2.27e-322
Time:                        19:45:47   Log-Likelihood:                -2105.4
No. Observations:                 311   AIC:                             4215.
Df Residuals:                     309   BIC:                             4222.
Df Model:                           1                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        602.3464     53.334     11.294      0.0